# FashionMNIST

In [9]:
import torch
from torch import nn

from torch.utils.data import DataLoader

from torchvision import datasets
from torchvision.transforms import ToTensor

Initialize DataLoaders

In [10]:
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

batch_size=64

train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

In [11]:
device = torch.accelerator.current_accelerator() if torch.accelerator.is_available() else "cpu"
print(f"Using {device} device")

Using mps device


Define network structure

In [12]:
class NeuralNetwork(nn.Module):
	def __init__(self):
		super().__init__()
		self.flatten = nn.Flatten()
		self.linear_relu_stack = nn.Sequential(
			nn.Linear(28 * 28, 512),
			nn.ReLU(),
			nn.Linear(512, 512),
			nn.ReLU(),
			nn.Linear(512, 10)
		)

	def forward(self, x):
		x = self.flatten(x)
		logits = self.linear_relu_stack(x)
		return logits

model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


Helper methods for training and testing

In [13]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)
        
        pred = model(X)
        loss = loss_fn(pred, y)
        
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"Loss {loss}, {current}/{size}")


def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    correct = 0

    model.eval()

    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)

            pred = model(X)
            correct += (pred.argmax(1) == y).sum().item() # changed
    
    accuracy = correct / size
    print(f"Accuracy {accuracy * 100}%")

Training and evaluation (with checkpointing)

In [14]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=5e-3)

epochs = 25
for e in range(1, epochs + 1):
    print(f"Epoch {e}")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)

    torch.save({
        "model_state_dict": model.state_dict(),
        "optimizer_state_dict": optimizer.state_dict(),
        "epoch": e
    }, "models/fashion_mnist_checkpoint.pth")

print("Done!")

Epoch 1
Loss 2.293273448944092, 64/60000
Loss 2.235753059387207, 6464/60000
Loss 2.1303515434265137, 12864/60000
Loss 2.057518482208252, 19264/60000
Loss 1.8624680042266846, 25664/60000
Loss 1.667799472808838, 32064/60000
Loss 1.5291783809661865, 38464/60000
Loss 1.315490961074829, 44864/60000
Loss 1.258042573928833, 51264/60000
Loss 1.1035103797912598, 57664/60000
Accuracy 65.29%
Epoch 2
Loss 1.168516993522644, 64/60000
Loss 1.115630865097046, 6464/60000
Loss 0.8857837915420532, 12864/60000
Loss 0.9927456378936768, 19264/60000
Loss 0.8557217121124268, 25664/60000
Loss 0.8570118546485901, 32064/60000
Loss 0.8965393900871277, 38464/60000
Loss 0.8217214345932007, 44864/60000
Loss 0.8368729948997498, 51264/60000
Loss 0.7918344140052795, 57664/60000
Accuracy 71.2%
Epoch 3
Loss 0.7929809093475342, 64/60000
Loss 0.8663868308067322, 6464/60000
Loss 0.6117939949035645, 12864/60000
Loss 0.8067803978919983, 19264/60000
Loss 0.7039192914962769, 25664/60000
Loss 0.695603609085083, 32064/60000
Loss

Saving the model

In [15]:
# save model
torch.save(model.state_dict(), "models/fashion_mnist.pth")
print("Saved PyTorch Model State to fashion_mnist.pth")

# load model
model = NeuralNetwork().to(device)
model.load_state_dict(torch.load("models/fashion_mnist.pth"))

Saved PyTorch Model State to fashion_mnist.pth


<All keys matched successfully>

In [17]:
classes = ["T-shirt/top", "Trouser", "Pullover", "Dress", "Coat", "Sandal", "Shirt", "Sneaker", "Bag", "Ankle boot"]

model.eval()
x, y = test_data[0][0], test_data[0][1]

with torch.no_grad():
    x = x.to(device)

    pred = model(x)
    predicted, actual = classes[pred.argmax(1)], classes[y]
    
    print(f'Predicted: "{predicted}", Actual: "{actual}"')

Predicted: "Ankle boot", Actual: "Ankle boot"
